In [ ]:
import time
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.datasets import fetch_20newsgroups
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Automatically download missing NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Check CUDA availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Custom print functions
def print_timer_info(message):
    print(f"[TIMER INFO] {message}")

def print_output_data(message):
    print(f"[OUTPUT DATA] {message}")

# Load the dataset
start_time = time.time()
print_timer_info("Loading the 20 Newsgroups dataset...")
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({'Text': newsgroups.data, 'Category': newsgroups.target})
df['Category Name'] = df['Category'].apply(lambda x: newsgroups.target_names[x])
print_timer_info(f"Time taken to load dataset: {time.time() - start_time:.2f} seconds")

# Pre-processing
print_timer_info("Starting pre-processing...")
preprocess_start_time = time.time()

df['Text'] = df['Text'].str.lower()  # Lowercasing
df['Text'] = df['Text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))  # Remove punctuation/special characters
df['Tokens'] = df['Text'].apply(word_tokenize)  # Tokenization

stop_words = set(stopwords.words('english'))
df['Tokens'] = df['Tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])  # Stopwords removal

stemmer = PorterStemmer()
df['Tokens'] = df['Tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])  # Stemming

print_timer_info(f"Total time for pre-processing: {time.time() - preprocess_start_time:.2f} seconds")

# Move processed data to GPU if available (Example - converting text data to tensor)
max_token_length = max(df['Tokens'].apply(len))
print_output_data(f"Maximum length of tokenized text: {max_token_length}")

# Data Preparation
data_prep_start_time = time.time()
print_timer_info("Starting data preparation...")

# Define and fit the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=6620, stop_words='english')  # Set max_features to limit vocab size if necessary
X_tfidf = tfidf_vectorizer.fit_transform(df['Text']).toarray()  # Assuming you are using the original 'Text' column

# Encode the labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Category'])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create PyTorch Dataset
class TfidfDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TfidfDataset(X_train_tensor, y_train_tensor)
val_dataset = TfidfDataset(X_val_tensor, y_val_tensor)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print_timer_info(f"Total time for data preparation: {time.time() - data_prep_start_time:.2f} seconds")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[TIMER INFO] Loading the 20 Newsgroups dataset...
[TIMER INFO] Time taken to load dataset: 13.14 seconds
[TIMER INFO] Starting pre-processing...
